In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important }</style>"))

In [2]:
from utilities import *
from models import *

import joblib

In [3]:
pin_file = "../Data/pin.csv"
pin = read_pin(pin_file)

In [4]:
enc = joblib.load("../Models/MLP_Classification_GroupBy_FFill_MinMax_Encoder.joblib")

In [5]:
model = MLPClassifier()
model.load('../Models/MLP_Classification_GroupBy_FFill_MinMax.h5')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1152      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 14)                462       
Total params: 3,694
Trainable params: 3,694
Non-trainable params: 0
_________________________________________________________________


In [6]:
client = InfluxDBClient()

In [7]:
data = client.retrieveData(seconds=3, beacon="0117C55D14E4")

In [8]:
data = formatData(data)

In [9]:
data[scanners] = minMaxScaling(data[scanners])

In [10]:
data = data.groupby(pd.Grouper(freq="1s")).mean().reset_index()

In [11]:
data = data.ffill().fillna(0)

In [12]:
pred = model.predict(data[scanners].values)

In [13]:
pred = np.argmax(pred, axis=1)

In [14]:
pred = enc.inverse_transform(pred)

In [15]:
x = np.vectorize(lambda x: pin[x]["X"])(pred)
y = np.vectorize(lambda x: pin[x]["Y"])(pred)

In [16]:
(x.mean(), y.mean())

(27.77232575, 11.609775)